## Visualization

In [1]:
!pip install geemap folium google-cloud-storage rasterio matplotlib localtileserver ipyleaflet > /dev/null 2>&1

In [2]:
import ee
import geemap
import os
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import rasterio
import folium
from folium.plugins import TimeSliderChoropleth
from branca.colormap import LinearColormap

drive.mount('/content/drive')
work_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/'
os.makedirs(work_dir, exist_ok=True)


Mounted at /content/drive


In [ ]:
ee.Authenticate()
ee.Initialize(project='manifest-pride-258211')

In [ ]:
# Drive'dan TIFF dosyalarını yükle
lst_2015 = os.path.join(work_dir, 'LST_2015_Hamburg_test.tif')
lst_2025 = os.path.join(work_dir, 'LST_2025_Hamburg_test.tif')

In [ ]:
def plot_tiff(path, title):
    with rasterio.open(path) as src:
        arr = src.read(1)  # İlk bandı oku
        plt.figure(figsize=(8,6))
        plt.imshow(arr, cmap='jet')
        plt.colorbar(label='LST Value')
        plt.title(title)
        plt.axis('off')
        plt.show()

# 2015 dosyasını çiz
plot_tiff(lst_2015, 'LST 2015 Hamburg')

# 2025 dosyasını çiz
plot_tiff(lst_2025, 'LST 2025 Hamburg')

In [ ]:
output_dir = '/content/drive/MyDrive/outputs'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
from localtileserver import TileClient, get_leaflet_tile_layer
from ipyleaflet import Map, LayersControl, WidgetControl
import ipywidgets as widgets
import threading
import time


# TileClient ile tile sunucusu oluştur
client_2015 = TileClient(lst_2015)
client_2025 = TileClient(lst_2025)

# Tile katmanları oluştur
layer_2015 = get_leaflet_tile_layer(client_2015)
layer_2025 = get_leaflet_tile_layer(client_2025)
layer_2025.opacity = 0  # Başta sadece 2015 görünsün

# Harita oluştur
m = Map(center=client_2015.center(), zoom=12)
m.add_layer(layer_2015)
m.add_layer(layer_2025)
m.add_control(LayersControl())

# Yıl slider
slider = widgets.SelectionSlider(
    options=['2015', '2025'],
    value='2015',
    description='Year:',
    layout={'width': '400px'}
)

def toggle_layer(year):
    if year == '2015':
        layer_2015.opacity = 1
        layer_2025.opacity = 0
    else:
        layer_2015.opacity = 0
        layer_2025.opacity = 1

def on_slider_change(change):
    toggle_layer(change['new'])

slider.observe(on_slider_change, names='value')
slider_control = WidgetControl(widget=slider, position='bottomright')
m.add_control(slider_control)

is_animating = False
animation_button = widgets.ToggleButton(
    value=False,
    description='▶️ Animate',
    tooltip='Play/Pause animation',
    layout=widgets.Layout(width='100px')
)

def run_animation():
    global is_animating
    years = ['2015', '2025']
    idx = years.index(slider.value)
    while is_animating:
        idx = (idx + 1) % len(years)
        slider.value = years[idx]
        time.sleep(1.5)  # geçiş süresi (saniye)

def on_button_toggle(change):
    global is_animating
    is_animating = change['new']
    if is_animating:
        animation_button.description = '⏸️ Pause'
        thread = threading.Thread(target=run_animation)
        thread.start()
    else:
        animation_button.description = '▶️ Animate'

animation_button.observe(on_button_toggle, names='value')
anim_control = WidgetControl(widget=animation_button, position='bottomleft')
m.add_control(anim_control)

output_path = os.path.join(output_dir, 'map.html')
m.save(output_path)
print(f"Harita kaydedildi: {output_path}")


Harita kaydedildi: /content/drive/MyDrive/outputs/map.html


In [ ]:
!pip install nbstripout > /dev/null 2>&1

In [ ]:
nbstripout /content/drive/MyDrive/notebooks/07_visualization_deploy.ipynb
